**1. 필요한 라이브러리 불러오기**

In [ ]:
!	pip install selenium==4.0.0   
!apt-get update 
!apt install chromium-chromedriver 
!cp /usr/lib/chromium-browser/chromedriver /usr/bin 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 954 kB 25.3 MB/s 
     |████████████████████████████████| 358 kB 58.1 MB/s 
     |████████████████████████████████| 139 kB 35.0 MB/s 
     |████████████████████████████████| 4.1 MB 52.3 MB/s 
     |████████████████████████████████| 55 kB 4.2 MB/s 
     |████████████████████████████████| 58 kB 6.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.11 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Get:1 http://

- colab에서 selenium을 실행하기 위해서는 위의 4가지 코드를 작성해야한다. 
- 셀레니움 최신 버전 4.0.0으로 설치했다.

In [ ]:
import time

#[1]
from openpyxl import Workbook #크롤링한 댓글 저장
import pandas as pd #크롤링한 댓글 저장

#[2]
from selenium import webdriver #댓글크롤링
from selenium.webdriver.common.keys import Keys #댓글크롤링
from bs4 import BeautifulSoup #댓글크롤링

#[3]
from urllib.request import urlopen #stiring이나 Request 객체인 URL 오픈
from urllib.parse import quote_plus #한글깨짐 현상 방지

#[4]
import warnings
warnings.filterwarnings('ignore')

- 유튜브 댓글 크롤링을 하기 위해 selenium과 BeautifulSoup를 사용했다. 

- 크롤링 한 댓글을 엑셀 파일에 저장하기 위해서 openpyxl과 pandas를 사용했다. 

**2. colab에서 웹브라우저 창 생성되지 않으므로 별도의 옵션 설정**

In [ ]:
#Colab에선 웹브라우저 창이 뜨지 않으므로 별도 설정한다.

options = webdriver.ChromeOptions()
options.add_argument('--headless')        # Head-less 설정
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', options=options)

- [headless]
별도의 GUI 창이 뜨지 않고 사이트에 접속해서 페이지를 받는등의 작업을 실행할 수 있다.

- [no-sandbox]
chrome의 sandbox는 멀웨어(악성 소프트웨어)를 방지하기 위해 사용된다. no-sandbox로 설정해 이런 기능을 off했다.

- [--disable-dev-shm-usage]
dev/shm 디렉토리를 사용하지 않는다. 이 디렉토리는 공유 메모리를 담당한다. 즉, 모든 사용자가 읽고 쓰지 못하도록 제한했다.

**3. 크롤링할 유튜브 불러오기**

In [ ]:
wb = Workbook(write_only=True)
ws = wb.create_sheet() #댓글 저장할 빈 엑셀파일 생성


url = "https://bit.ly/3JjNkzt" #해당 url로 이동
driver.get(url)
driver.implicitly_wait(3)

time.sleep(1.5)

driver.execute_script("window.scrollTo(0,800)")
time.sleep(3)

**4. 웹페이지 끝까지 스크롤하기(무한 스크롤)**

In [ ]:
last_height = driver.execute_script("return document.documentElement.scrollHeight") #마지막 시점의 창 높이 저장

while True:
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);") #창 높이까지 스크롤
    time.sleep(1.5) #스크롤 후 창이 로딩될 때까지 1.5초 기다리겠다는 명령어

    new_height = driver.execute_script("return document.documentElement.scrollHeight") #스크롤이 된 후의 창 높이를 새로운 높이로 저장
    if new_height == last_height: #새로운 높이가 이전 높이와 변하지 않았으면 스크롤 종료
        break
    last_height = new_height

time.sleep(1.5)

- 유튜브 영상 내 등록된 모든 댓글을 노출되게 만들기 위해 페이지를 끝까지 스크롤 될 수 있도록 한다.

**5. 유튜브 팝업 닫기**

In [ ]:
from selenium.webdriver.common.by import By

try:
 driver.find_elements(By.CSS_SELECTOR,"dismiss-button > a").click()
except:
  pass  

- 유튜브 프리미엄 사용을 유도하는 팝업창을 닫아준다.

- 이 팝업창을 닫아주지 않으면 추후 댓글을 가져오는데 방해가 될 수 있어 꼭 닫아주는 것이 좋다. 


- [from selenium.webdriver.common.by import By]
셀레니움 버전 4로 업데이트 됨에 따라 이 라이브러리를 추가하였다.

**6. 대댓글 버튼 누르기**

In [ ]:
buttons = driver.find_elements(By.CSS_SELECTOR,"dismiss-button > a")

time.sleep(1.5)

for button in buttons:
  button.send_keys(Keys.ENTER)
  time.sleep(1.5)
  button.click()

- 유튜브 댓글은 댓글 뿐만 아니라 대댓글까지 존재한다. 일반적으로 토글 형태로 저장되어 있어서 이 토글 버튼을 모두 눌러줘야 모든 댓글을 불러올 수 있다.

- 처음에 driver.find_element_by_css_selector 함수로 작성했었는데 오류가 났다. 찾아보니 셀레니움 버전이 4로 업데이트 되어 이에 따라 사용하는 함수명이 변경되었다. 따라서 셀레니움 버전을 4.0.0 으로 설치해주었고, 함수명도 "driver.find_elements"로 변경해주었다.

**7. 댓글 가져오기**

In [ ]:
html_source = driver.page_source  
soup = BeautifulSoup(html_source, 'html.parser')

id_list = soup.select("div#header-author > h3 > #author-text > span")
comment_list = soup.select("yt-formatted-string#content-text")

id_final = []
comment_final = []

for i in range(len(comment_list)):
    temp_id = id_list[i].text
    temp_id = temp_id.replace('\n', '') #줄바꿈
    temp_id = temp_id.replace('\t', '') #tab
    temp_id = temp_id.replace('    ', '')
    id_final.append(temp_id) # 댓글 작성자

    temp_comment = comment_list[i].text
    temp_comment = temp_comment.replace('\n', '')
    temp_comment = temp_comment.replace('\t', '')
    temp_comment = temp_comment.replace('    ', '')
    comment_final.append(temp_comment) # 댓글 내용

- 유튜브 댓글을 작성한 사람의 아이디와, 작성한 댓글 내용 2가지로 구분하여 가져온다. 

- parser : Compiler나 Interpreter에서 원시 프로그램을 읽어 들여 그 문장의 구조를 알아내는 구문분석(parsing)을 행하는 프로그램 

**8. 댓글 저장하기**

In [ ]:
pd_data = {"아이디" : id_final , "댓글 내용" : comment_final}
youtube_pd = pd.DataFrame(pd_data)

youtube_pd.to_excel('힙으뜸_220724.xlsx')

pd_data = './gdrive/My Drive/Colab Notebooks/crawling/'

- 판다스를 사용해 추출한 유튜브 댓글 아이디와 댓글 내용을 데이터프레임 형태로 저장한다.

- 저장한 데이터프레임을 엑셀 파일로 저장해주면 유튜브 댓글 크롤링 작업이 완료된다.